# Preparation
## Paths

In [ ]:
# Import Operating System
import os

In [ ]:
ipynb_path = os.path.dirname(os.path.realpath("__file__"))
print(ipynb_path)

In [ ]:
root = "\\".join(ipynb_path.split("\\")[:-2])+"\\datasets"
print(root)

In [ ]:
workspace = "custom"

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join(root, workspace),
    'OVERLAYS_PATH': os.path.join(root, workspace,'overlays'),
    'BACKGROUNDS_PATH': os.path.join(root, workspace,'backgrounds'),
    'IMAGES_PATH': os.path.join(root, workspace,'images'),
    'TEST_IMAGES_PATH': os.path.join(root, workspace ,'images', 'test'),
    'TRAIN_IMAGES_PATH': os.path.join(root, workspace ,'images', 'train'),
    'VAL_IMAGES_PATH': os.path.join(root, workspace ,'images', 'val'),
    'ANNOTATIONS_PATH': os.path.join(root, workspace,'annotations'),
    'LABELS_PATH': os.path.join(root, workspace,'labels'),
    'TEST_LABELS_PATH': os.path.join(root, workspace, 'labels', 'test'),
    'TRAIN_LABELS_PATH': os.path.join(root, workspace, 'labels', 'train'),
    'VAL_LABELS_PATH': os.path.join(root, workspace, 'labels', 'val'),
 }
print(paths)

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## Backgrounds

In [ ]:
_, _, background_files = next(os.walk(paths['BACKGROUNDS_PATH']), (None, [], []))
print("background files: " + str(len(background_files)))

## Overlays

In [ ]:
_, overlays_folders, overlays_files = next(os.walk(paths['OVERLAYS_PATH']), (None, [], []))
print("overlays folders: " + str(len(overlays_folders)))
print("overlays files: " + str(len(overlays_files)))

## Labels

In [ ]:
labels = []
overlays = []
for folder in overlays_folders:
    #print("label: " + folder)
    labels.append(folder)
    _, _, overlays_files_versions = next(os.walk(os.path.join(paths['OVERLAYS_PATH'], folder)), (None, [], []))
    for file in overlays_files_versions:
        #print("file: " + file)
        overlays.append(os.path.join(paths['OVERLAYS_PATH'], folder, file))

for file in overlays_files:
    overlays.append(os.path.join(paths['OVERLAYS_PATH'], file))
    file_label = "-".join(file.split("-")[:-1])
    #print("label: " + file_label)
    labels.append(file_label)

unique_labels = list(dict.fromkeys(labels))
print(labels)
print(unique_labels)
#print(overlays)

list_filename = os.path.join(paths['WORKSPACE_PATH'], 'className.list')
#print(list_filename)

with open(list_filename, 'w') as file :
    file.write('')

for (l, label) in enumerate(unique_labels):
    with open(list_filename, 'a') as file :
        file.write(label+"\n")

In [ ]:
#!pip install opencv-python
#!pip install matplotlib
#!pip install scipy
#!pip install imutils

In [ ]:
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import math
import random

In [ ]:
def crop_alpha(image_to_crop):
    image_to_crop[image_to_crop < 0.008] = 0.
    # axis 0 is the row(y) and axis(x) 1 is the column
    # get the nonzero alpha coordinates
    y, x = image_to_crop[:, :, 3].nonzero()
    minx = np.min(x)
    miny = np.min(y)
    maxx = np.max(x)
    maxy = np.max(y)
    croped = image_to_crop[miny:maxy, minx:maxx]

    croped[croped < 0] = 0.
    croped[croped > 1] = 1.
    return croped

In [ ]:
def rotate_image(image_to_rotate, rmax=45, rmin=0):
    deg = random.uniform(rmin, rmax)
    pos = 2*(random.random()-0.5)
    
    height, width = image_to_rotate.shape[:2]
    dia = int(math.sqrt(math.pow(height, 2) + math.pow(width, 2)))

    n_channels = 4
    transparent_img = np.zeros((dia, dia, n_channels), dtype=np.single)

    # padding_y
    bpy = int((dia - height)/2)
    #print((dia - height),bpy)
    # padding_x
    bpx = int((dia - width)/2)
    #print((dia - width),bpx)

    transparent_img[bpy:(bpy+height), bpx:(bpx+width)] = image_to_rotate

    rotated = imutils.rotate(transparent_img, angle=deg*pos)
    return crop_alpha(rotated)

In [ ]:
def adjust_image(image_to_adjust, gmin=0.5, gmax=1.5, smin=0.5, smax=1.0):
    saturation = random.uniform(smin, smax)
    gamma = random.uniform(gmin, gmax)

    a = rotated[:, :, 3]

    hsv = cv2.cvtColor(image_to_adjust, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    s_desat = cv2.multiply(s, saturation)
    v_gamma = cv2.multiply(v, gamma)
    hsv_new = cv2.merge([h, s_desat, v_gamma])

    bgr_adjust = cv2.cvtColor(hsv_new, cv2.COLOR_HSV2BGR)
    adjusted = cv2.cvtColor(bgr_adjust, cv2.COLOR_RGB2RGBA)
    adjusted[:, :, 3] = a
    adjusted[adjusted < 0] = 0.
    adjusted[adjusted > 1] = 1.
    return adjusted

In [ ]:
def resize_image(image_to_resize, max=50, min=0):
    resize = random.uniform(min, max)
    procent = 1-resize/100

    height, width = image_to_resize.shape[:2]
    dim = (int(width*procent), height)
    resized = cv2.resize(image_to_resize, dim, interpolation=cv2.INTER_AREA)
    return resized

In [ ]:
def resize_if_big(image_to_resize, size=100):
    height, width = image_to_resize.shape[:2]

    dim = (width, height)
    if height > size:
        dim = (int(size*width/height), size)

    if width > height:
        if width > size:
            dim = (size, int(size*height/width))
    if dim == (width, height):
        return image_to_resize
    resized = cv2.resize(image_to_resize, dim, interpolation=cv2.INTER_AREA)
    return resized

In [ ]:
def resize_if_small(image_to_resize, size=100):
    height, width = image_to_resize.shape[:2]

    dim = (width, height)
    if height < size:
        dim = (int(size*width/height), size)

    if width < height:
        if width < size:
            dim = (size, int(size*height/width))
    if dim == (width, height):
        return image_to_resize
    resized = cv2.resize(image_to_resize, dim, interpolation=cv2.INTER_AREA)
    return resized

In [ ]:
def random_crop(image, crop_height=320, crop_width=320, padding=0):
    max_x = image.shape[1]-crop_width-padding*2
    max_y = image.shape[0]-crop_height-padding*2
    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)
    crop = image[y: y+crop_height+padding*2, x: x+crop_width+padding*2]
    return crop

In [ ]:
def combine_images(overlay, background):
    combine_image = background
    overlay_height, overlay_width = overlay.shape[:2]
    background_height, background_width = background.shape[:2]
    # background_padding_y
    bpy = int((background_height - overlay_height)/2)
    # background_padding_x
    bpx = int((background_width - overlay_width)/2)
    for y in range(overlay_height):
        for x in range(overlay_width):
            overlay_color = overlay[y, x, :3]
            # 4th element is the alpha channel, convert from 0-255 to 0.0-1.0
            overlay_alpha = overlay[y, x, 3]
            background_color = background[bpy+y, bpx+x]
            combine_color = background_color * \
                (1 - overlay_alpha)+overlay_color*overlay_alpha
            # update the background image in place
            combine_image[bpy+y, bpx+x] = combine_color

    combine_image[combine_image < 0] = 0.
    combine_image[combine_image > 1] = 1.
    
    return combine_image

In [ ]:
print("overlays files: " + str(len(overlays)))
print(overlays[0])
print("background files: " + str(len(background_files)))
background = os.path.join(paths['BACKGROUNDS_PATH'], background_files[0])
print(background)

In [ ]:
def create_label(classID, filePath, w=1.0, h=1.0, x=0.5, y=0.5):
    #print(f"filePath: {filePath}")
    labelText = str(classID)+" "+str(x)+" "+str(y)+" "+str(w)+" "+str(h)+"\n"
    #print(labelText)
    with open(filePath, 'w') as file :
        file.write(labelText)

In [ ]:
def create_img_and_label(b,o,combine,w,h,x,y,sub_folder='train'):
    b_part = b.split(".")[0]
    #print(b+' -> '+b_part)

    o_part = o.split(".")[0]
    o_part = o_part.split("\\")
    if len(o_part[-1].split("-")) > 1:
        o_part = o_part[-1]
    else:
        o_part = o_part[-2]+"-"+o_part[-1]
    #print(o+' -> '+o_part)

    class_name = "-".join(o_part.split("-")[:2])
    classID = unique_labels.index(class_name)
    #print('Class Name: '+str(classID)+'. '+class_name)
    
    new_file_name = o_part+'-b'+b_part
    #print('New file name: '+new_file_name)
    
    img_filePath = os.path.join(paths['IMAGES_PATH'], sub_folder, new_file_name+'.png')
    label_filePath = os.path.join(paths['LABELS_PATH'], sub_folder, new_file_name+'.txt')
    #print(img_filePath,label_filePath)

    create_label(classID, label_filePath, w, h, x, y)
    cv2.imwrite(img_filePath, cv2.cvtColor(combine.astype('uint8'), cv2.COLOR_BGR2RGB))

In [ ]:
def test_random():
    finish_size = 32
    cols = 5
    rows = 5
    i=0
    fig = plt.figure(figsize=(20,21))

    b = background_files[np.random.randint(0, len(background_files)-1)]
    o = overlays[np.random.randint(0, len(overlays)-1)]

    background = os.path.join(paths['BACKGROUNDS_PATH'], b)

    bgi = mpimg.imread(background)
    bgi = resize_if_small(bgi, size=finish_size)
    bg_crop = random_crop(bgi, finish_size, finish_size, 0)

    #imgplot = plt.imshow(bg_crop)
    fig.add_subplot(rows, cols, i+1)
    plt.imshow(bg_crop, alpha=1)
    i+=1

    img = mpimg.imread(o)

    #imgplot = plt.imshow(bg_crop)
    fig.add_subplot(rows, cols, i+1)
    plt.imshow(img, alpha=1)
    i+=1

    #cropImg = crop_alpha(img)

    #fig.add_subplot(rows, cols, i+1)
    #plt.imshow(cropImg, alpha=1)
    #i+=1

    rotated = rotate_image(img)

    fig.add_subplot(rows, cols, i+1)
    plt.imshow(rotated, alpha=1)
    i+=1

    adjusted = adjust_image(rotated)

    fig.add_subplot(rows, cols, i+1)
    plt.imshow(adjusted, alpha=1)
    i+=1

    resized = resize_image(adjusted)

    fig.add_subplot(rows, cols, i+1)
    plt.imshow(resized, alpha=1)
    i+=1

    #overlay = crop_alpha(resized)
    overlay = resize_if_small(resized, finish_size)
    overlay = resize_if_big(overlay, finish_size)

    fig.add_subplot(rows, cols, i+1)
    plt.imshow(overlay, alpha=1)
    i+=1

    combine = (combine_images(overlay, bg_crop/255)*255).astype('uint8')

    fig.add_subplot(rows, cols, i+1)
    plt.imshow(combine, alpha=1)
    i+=1

    combine[combine < 0] = 0
    combine[combine > 255] = 255

    h, w = overlay.shape[:2]
    combine_h, combine_w = combine.shape[:2]
    #print(h, w, combine_h, combine_w)

    #create_img_and_label(b, o, combine, w=w/combine_w, h=h/combine_h, x=0.5, y=0.5)

    plt.show()
    #imgplot = plt.imshow(combine)

In [ ]:
finish_size = 32

len_b = len(background_files)
len_o = len(overlays)

cols = 5
rows = 5

## Train

In [ ]:
fig = plt.figure(figsize=(20,21))
# ax enables access to manipulate each of subplots
ax = []

i = 0
for b in background_files:
    background = os.path.join(paths['BACKGROUNDS_PATH'], b)

    bgi = mpimg.imread(background)
    bgi = resize_if_small(bgi, size=finish_size)

    for o in overlays:
        if i == rows*cols:
            print('_')
            #break

        bg_crop = random_crop(bgi, finish_size, finish_size, 0)
        #imgplot = plt.imshow(bg_crop)
        
        img = mpimg.imread(o)
        img = resize_if_small(img, finish_size)
        
        #cropImg = crop_alpha(img)
        rotated = rotate_image(img)
        adjusted = adjust_image(rotated)
        resized = resize_image(adjusted)

        overlay = crop_alpha(resized)
        overlay = resize_if_small(overlay, finish_size)
        overlay = resize_if_big(overlay, finish_size)

        combine = combine_images(overlay, bg_crop/255)
        combine = (combine*255).astype('uint8')
        combine[combine < 0] = 0
        combine[combine > 255] = 255
        
        h, w = overlay.shape[:2]
        combine_h, combine_w = combine.shape[:2]
        
        create_img_and_label(b, o, combine, w/combine_w, h/combine_h, 0.5, 0.5)
        
        if i < rows*cols:
            ax.append( fig.add_subplot(rows, cols, i+1) )
            ax[-1].set_title("ax:"+str(i)+" "+str(h)+" "+str(w))  # set title
            plt.imshow(combine, alpha=1)

        i+=1

plt.show()

## Test

In [ ]:
finish_size = 320
fig = plt.figure(figsize=(20,21))
# ax enables access to manipulate each of subplots
ax = []

i = 0
for o in overlays:

    if i == rows*cols:
        print('_')
        #break

    b = background_files[np.random.randint(0, len(background_files)-1)]
    background = os.path.join(paths['BACKGROUNDS_PATH'], b)

    bgi = mpimg.imread(background)
    bgi = resize_if_small(bgi, size=finish_size)

    #bg_crop = random_crop(bgi, finish_size, finish_size, 0)
    #imgplot = plt.imshow(bg_crop)

    img = mpimg.imread(o)
    img = resize_if_small(img, finish_size)

    #cropImg = crop_alpha(img)
    rotated = rotate_image(img)
    adjusted = adjust_image(rotated)
    resized = resize_image(adjusted)

    overlay = crop_alpha(resized)
    overlay = resize_if_small(overlay, finish_size)
    overlay = resize_if_big(overlay, finish_size)

    combine = combine_images(overlay, bgi/255)
    combine = (combine*255).astype('uint8')
    combine[combine < 0] = 0
    combine[combine > 255] = 255

    h, w = overlay.shape[:2]
    c_h, c_w = combine.shape[:2]

    create_img_and_label(b, o, combine, w/c_w, h/c_h, 0.5, 0.5, 'test')

    if i < rows*cols:
        ax.append( fig.add_subplot(rows, cols, i+1) )
        ax[-1].set_title("test:"+str(i)+" "+str(h)+" "+str(w)+" "+str(c_h)+" "+str(c_w))  # set title
        plt.imshow(combine, alpha=1)

    i+=1

plt.show()

## Val

In [ ]:
finish_size = 640
fig = plt.figure(figsize=(20,21))
# ax enables access to manipulate each of subplots
ax = []

i = 0
for o in overlays:

    if i == rows*cols:
        print('_')
        #break

    b = background_files[np.random.randint(0, len(background_files)-1)]
    background = os.path.join(paths['BACKGROUNDS_PATH'], b)

    bgi = mpimg.imread(background)
    bgi = resize_if_small(bgi, size=finish_size)

    #bg_crop = random_crop(bgi, finish_size, finish_size, 0)
    #imgplot = plt.imshow(bg_crop)

    img = mpimg.imread(o)
    img = resize_if_small(img, finish_size)

    #cropImg = crop_alpha(img)
    rotated = rotate_image(img)
    adjusted = adjust_image(rotated)
    resized = resize_image(adjusted)

    overlay = crop_alpha(resized)
    overlay = resize_if_small(overlay, finish_size)
    overlay = resize_if_big(overlay, finish_size)

    combine = combine_images(overlay, bgi/255)
    combine = (combine*255).astype('uint8')
    combine[combine < 0] = 0
    combine[combine > 255] = 255

    h, w = overlay.shape[:2]
    c_h, c_w = combine.shape[:2]

    create_img_and_label(b, o, combine, w/c_w, h/c_h, 0.5, 0.5, 'val')

    if i < rows*cols:
        ax.append( fig.add_subplot(rows, cols, i+1) )
        ax[-1].set_title("val:"+str(i)+" "+str(h)+" "+str(w)+" "+str(c_h)+" "+str(c_w))  # set title
        plt.imshow(combine, alpha=1)

    i+=1

plt.show()

## data yaml

usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP] [--epochs EPOCHS]
                [--batch-size BATCH_SIZE] [--img-size IMG_SIZE [IMG_SIZE ...]] [--rect] [--resume [RESUME]] [--nosave]
                [--notest] [--noautoanchor] [--evolve] [--bucket BUCKET] [--cache-images] [--image-weights]
                [--device DEVICE] [--multi-scale] [--single-cls] [--adam] [--sync-bn] [--local_rank LOCAL_RANK]
                [--workers WORKERS] [--project PROJECT] [--entity ENTITY] [--name NAME] [--exist-ok] [--quad]
                [--linear-lr] [--label-smoothing LABEL_SMOOTHING] [--upload_dataset] [--bbox_interval BBOX_INTERVAL]
                [--save_period SAVE_PERIOD] [--artifact_alias ARTIFACT_ALIAS]

In [ ]:
nc = len(unique_labels)
dataYamlFile = workspace+'-'+str(nc)+'-data.yaml'
dataYamlPath = os.path.join(paths['WORKSPACE_PATH'], dataYamlFile)
print(f"filePath: {dataYamlPath}")
yamlText = '# python train.py --weights yolov7.pt --cfg cfg/training/yolov7-tiny.yaml --data '+dataYamlFile+' --hyp data/hyp.yaml --img-size 320 --device cpu --name '+workspace+'-'+str(nc)+'\n'
yamlText += '# python train.py --resume True --name '+workspace+'-'+str(nc)+'\n'
yamlText += 'train: '+paths['TRAIN_IMAGES_PATH']+'\n'
yamlText += 'val: '+paths['VAL_IMAGES_PATH']+'\n'
yamlText += 'test: '+paths['TEST_IMAGES_PATH']+'\n'
yamlText += '\n'
yamlText += '# number of classes\n'
yamlText += 'nc: '+str(nc)+'\n'
yamlText += '\n'
yamlText += '# class names\n'
yamlText += 'names: ["'+'", "'.join(unique_labels)+'"]\n'
print(yamlText)
with open(dataYamlPath, 'w') as file:
    file.write(yamlText)